In [1]:
# Preparación de los dos conjuntos:

In [2]:
import os
import shutil

# Rutas base
base_dud = "../data/raw/dud"
base_acs = "../data/raw/acs/"
output_dud = "../data/process/dud/"
output_acs = "../data/process/acs/"

# Asegurar que las carpetas de salida existan
os.makedirs(output_dud, exist_ok=True)
os.makedirs(output_acs, exist_ok=True)

# Subcarpetas que se deben explorar
subdirs = ["n1", "n2"]

paired_count = 0
missing_count = 0

for subdir in subdirs:
    dud_dir = os.path.join(base_dud, subdir)
    acs_dir = os.path.join(base_acs, subdir)
    
    if not os.path.isdir(dud_dir) or not os.path.isdir(acs_dir):
        print(f"Advertencia: faltan carpetas en {subdir}")
        continue
    
    dud_files = [f for f in os.listdir(dud_dir) if f.endswith(".fits")]
    
    for dud_file in dud_files:
        # Construir el nombre correspondiente en ACS
        base_name = dud_file.replace(".fits", "")
        acs_file = base_name + "_ACS.fits"

        dud_path = os.path.join(dud_dir, dud_file)
        acs_path = os.path.join(acs_dir, acs_file)

        if os.path.exists(acs_path):
            # Copiar ambos archivos a su nueva ubicación
            shutil.copy2(dud_path, os.path.join(output_dud, dud_file))
            shutil.copy2(acs_path, os.path.join(output_acs, acs_file))
            paired_count += 1
        else:
            missing_count += 1

print(f"\nProceso completado.")
print(f"Pares encontrados y copiados: {paired_count}")
print(f"Archivos DUD sin par ACS: {missing_count}")



Proceso completado.
Pares encontrados y copiados: 16114
Archivos DUD sin par ACS: 5886


# Procesado y normalizacion de los datos para su uso por un modelo
No se ha hecho, pero estaria bien:  (Si se ha hecho una normalización global)

📉 3. Normalización estadística por imagen  

Justificación: La normalización min-max global es buena para mantener una escala uniforme, pero si hay alta variabilidad entre imágenes, podrías experimentar con:  
- Normalización por imagen individual usando media y desviación típica (z-score).  
- O bien escalar entre 0 y 1 por imagen, especialmente útil si el rango dinámico varía mucho entre ejemplos.  

In [1]:
import os
import numpy as np
from astropy.io import fits
from tqdm import tqdm

# Directorios de entrada
dud_dir = "../data/process/dud/"
acs_dir = "../data/process/acs/"

# Directorios de salida
output_base = "../data/log/normalized/"
dud_output_dir = os.path.join(output_base, "dud")
acs_output_dir = os.path.join(output_base, "acs")
os.makedirs(dud_output_dir, exist_ok=True)
os.makedirs(acs_output_dir, exist_ok=True)

def normalize_log_image(image_data):
    image_data_float = image_data.astype(np.float32)
    log_transformed_data = np.log1p(image_data_float)

    min_val = np.min(log_transformed_data)
    max_val = np.max(log_transformed_data)

    if max_val == min_val:
        return np.zeros_like(log_transformed_data)
    
    normalized_data = (log_transformed_data - min_val) / (max_val - min_val)
    
    return normalized_data

dud_files = [f for f in os.listdir(dud_dir) if f.endswith(".fits")]
paired_count = 0

print("Normalizando logarítmicamente y guardando como .npy en carpetas separadas...")
for dud_file in tqdm(dud_files):
    base_name = dud_file.replace(".fits", "")
    acs_file = base_name + "_ACS.fits"

    dud_path = os.path.join(dud_dir, dud_file)
    acs_path = os.path.join(acs_dir, acs_file)

    if not os.path.exists(acs_path):
        continue

    try:
        with fits.open(dud_path, verify='silentfix') as dud_hdul:
            dud_data = next(hdu.data for hdu in dud_hdul if hdu.data is not None)
            dud_data = normalize_log_image(dud_data)

        with fits.open(acs_path, verify='silentfix') as acs_hdul:
            acs_data = next(hdu.data for hdu in acs_hdul if hdu.data is not None)
            acs_data = normalize_log_image(acs_data)

        np.save(os.path.join(dud_output_dir, base_name + ".npy"), dud_data)
        np.save(os.path.join(acs_output_dir, base_name + ".npy"), acs_data)

        paired_count += 1

    except Exception as e:
        print(f"Error procesando {dud_file}: {e}")

print(f"\n✅ Total de pares procesados y guardados: {paired_count}")

Normalizando logarítmicamente y guardando como .npy en carpetas separadas...


  1%|▊                                                                            | 162/16114 [00:00<01:34, 168.23it/s]C:\Users\Miguel\AppData\Local\Temp\ipykernel_21144\328024152.py:19: RuntimeWarning: invalid value encountered in log1p
  log_transformed_data = np.log1p(image_data_float)
100%|███████████████████████████████████████████████████████████████████████████| 16114/16114 [01:40<00:00, 159.98it/s]


✅ Total de pares procesados y guardados: 16114


Sí, otra opción que a menudo se considera superior para el downsampling (reducción de tamaño) en términos de calidad de imagen es la interpolación Lanczos.

Lanczos tiende a producir resultados más nítidos y con menos desenfoque que la bicúbica, lo que puede ser beneficioso para preservar los detalles finos en las galaxias. Sin embargo, a veces puede introducir un efecto de "ringing" (artefactos de anillo) alrededor de bordes muy contrastados, aunque esto rara vez es un problema mayor para la mayoría de las imágenes.

In [5]:
# pading y resice de las grandes
import os
import numpy as np
from astropy.io import fits # Todavía se usa si quieres el header, pero no para leer la imagen
from scipy.ndimage import zoom
from tqdm import tqdm

def process_normalized_npy(input_npy_dir, output_npy_dir, target_resolution):
    """
    Aplica padding a imágenes .npy pequeñas y reescalado Lanczos a imágenes .npy grandes
    para alcanzar una resolución cuadrada objetivo. Guarda las imágenes resultantes
    como archivos .npy.

    Args:
        input_npy_dir (str): Directorio de entrada con las imágenes .npy normalizadas.
        output_npy_dir (str): Directorio donde se guardarán las imágenes .npy procesadas.
        target_resolution (int): La resolución cuadrada deseada (ej. 21 para 21x21).
    """
    if not os.path.exists(input_npy_dir):
        print(f"Error: El directorio de entrada '{input_npy_dir}' no existe.")
        return
    if not os.path.exists(output_npy_dir):
        os.makedirs(output_npy_dir)
        print(f"Directorio de salida '{output_npy_dir}' creado.")

    npy_files = [f for f in os.listdir(input_npy_dir) if f.endswith(".npy")]

    if not npy_files:
        print(f"No se encontraron archivos .npy en '{input_npy_dir}'.")
        return

    print(f"\nProcesando imágenes .npy en '{input_npy_dir}' a {target_resolution}x{target_resolution} (padding/reescalado Lanczos)...")

    for filename in tqdm(npy_files):
        input_path = os.path.join(input_npy_dir, filename)
        output_path = os.path.join(output_npy_dir, filename) # Se mantiene el mismo nombre

        try:
            original_data = np.load(input_path)
            original_height, original_width = original_data.shape
            original_dtype = original_data.dtype # Usamos el dtype original, que será float32

            processed_data = np.zeros((target_resolution, target_resolution), dtype=original_dtype)

            # --- Lógica de procesamiento ---
            if original_width < target_resolution or original_height < target_resolution:
                # Aplicar padding para imágenes pequeñas
                start_y = (target_resolution - original_height) // 2
                start_x = (target_resolution - original_width) // 2
                
                processed_data[start_y : start_y + original_height,
                               start_x : start_x + original_width] = original_data
            else:
                # Reescalar Lanczos para imágenes grandes o de tamaño similar
                zoom_factor_h = target_resolution / original_height
                zoom_factor_w = target_resolution / original_width

                # No necesitamos convertir a float32 aquí porque ya están normalizadas (float32)
                zoomed_data = zoom(original_data, (zoom_factor_h, zoom_factor_w), order=5) 
                
                # Aseguramos que la imagen reescalada tenga exactamente la resolución objetivo
                current_h, current_w = zoomed_data.shape
                
                if current_h != target_resolution or current_w != target_resolution:
                    final_zoomed_data = np.zeros((target_resolution, target_resolution), dtype=original_dtype)
                    
                    insert_h = min(target_resolution, current_h)
                    insert_w = min(target_resolution, current_w)

                    start_y_zoom = (target_resolution - insert_h) // 2
                    start_x_zoom = (target_resolution - insert_w) // 2

                    src_start_y = (current_h - insert_h) // 2
                    src_start_x = (current_w - insert_w) // 2

                    final_zoomed_data[start_y_zoom : start_y_zoom + insert_h,
                                      start_x_zoom : start_x_zoom + insert_w] = \
                                      zoomed_data[src_start_y : src_start_y + insert_h,
                                                  src_start_x : src_start_x + insert_w]
                    processed_data = final_zoomed_data.astype(original_dtype)
                else:
                    processed_data = zoomed_data.astype(original_dtype)
            
            # Guardar el archivo .npy procesado
            np.save(output_path, processed_data)

        except Exception as e:
            print(f"Error procesando {filename} (reescalado Lanczos): {e}")

# Directorios de salida para los datos normalizados (.npy)
normalized_base_dir = "../data/log/normalized/"
dud_normalized_dir = os.path.join(normalized_base_dir, "dud")
acs_normalized_dir = os.path.join(normalized_base_dir, "acs")

# Define los directorios de salida para las imágenes .npy procesadas
# (después de normalización y reescalado/padding)
processed_dud_npy_dir = os.path.join(normalized_base_dir, "dud_processed_lanczos")
processed_acs_npy_dir = os.path.join(normalized_base_dir, "acs_processed_lanczos")

# Definir resoluciones objetivo (las mismas que antes)
target_dud_resolution = 96
target_acs_resolution = 384

print("\n--- PASO 2: Aplicando Padding/Reescalado Lanczos a datos .npy normalizados ---")

# Procesar las imágenes DUD normalizadas
process_normalized_npy(dud_normalized_dir, processed_dud_npy_dir, target_dud_resolution)

# Procesar las imágenes ACS normalizadas
process_normalized_npy(acs_normalized_dir, processed_acs_npy_dir, target_acs_resolution)

print("\n🎉 ¡Procesamiento completo: normalización y reescalado Lanczos aplicados!")
print(f"Imágenes DUD procesadas guardadas en: {processed_dud_npy_dir}")
print(f"Imágenes ACS procesadas guardadas en: {processed_acs_npy_dir}")


--- PASO 2: Aplicando Padding/Reescalado Lanczos a datos .npy normalizados ---
Directorio de salida '../data/log/normalized/dud_processed_lanczos' creado.

Procesando imágenes .npy en '../data/log/normalized/dud' a 96x96 (padding/reescalado Lanczos)...


100%|███████████████████████████████████████████████████████████████████████████| 16114/16114 [00:17<00:00, 937.19it/s]


Directorio de salida '../data/log/normalized/acs_processed_lanczos' creado.

Procesando imágenes .npy en '../data/log/normalized/acs' a 384x384 (padding/reescalado Lanczos)...


100%|███████████████████████████████████████████████████████████████████████████| 16114/16114 [00:27<00:00, 590.50it/s]


🎉 ¡Procesamiento completo: normalización y reescalado Lanczos aplicados!
Imágenes DUD procesadas guardadas en: ../data/log/normalized/dud_processed_lanczos
Imágenes ACS procesadas guardadas en: ../data/log/normalized/acs_processed_lanczos


In [2]:
# Solo pading
import os
import numpy as np
#from astropy.io import fits # Todavía se usa si quieres el header, pero no para leer la imagen
from scipy.ndimage import zoom
from tqdm import tqdm

def process_normalized_npy(input_npy_dir, output_npy_dir, target_resolution):
    """
    Aplica padding a imágenes .npy pequeñas y reescalado Lanczos a imágenes .npy grandes
    para alcanzar una resolución cuadrada objetivo. Guarda las imágenes resultantes
    como archivos .npy.

    Args:
        input_npy_dir (str): Directorio de entrada con las imágenes .npy normalizadas.
        output_npy_dir (str): Directorio donde se guardarán las imágenes .npy procesadas.
        target_resolution (int): La resolución cuadrada deseada (ej. 21 para 21x21).
    """
    if not os.path.exists(input_npy_dir):
        print(f"Error: El directorio de entrada '{input_npy_dir}' no existe.")
        return
    if not os.path.exists(output_npy_dir):
        os.makedirs(output_npy_dir)
        print(f"Directorio de salida '{output_npy_dir}' creado.")

    npy_files = [f for f in os.listdir(input_npy_dir) if f.endswith(".npy")]

    if not npy_files:
        print(f"No se encontraron archivos .npy en '{input_npy_dir}'.")
        return

    print(f"\nProcesando imágenes .npy en '{input_npy_dir}' a {target_resolution}x{target_resolution} (padding/reescalado Lanczos)...")

    for filename in tqdm(npy_files):
        input_path = os.path.join(input_npy_dir, filename)
        output_path = os.path.join(output_npy_dir, filename) # Se mantiene el mismo nombre

        try:
            original_data = np.load(input_path)
            original_height, original_width = original_data.shape
            original_dtype = original_data.dtype # Usamos el dtype original, que será float32

            processed_data = np.zeros((target_resolution, target_resolution), dtype=original_dtype)

            # --- Lógica de procesamiento ---
            if original_width < target_resolution or original_height < target_resolution:
                # Aplicar padding para imágenes pequeñas
                start_y = (target_resolution - original_height) // 2
                start_x = (target_resolution - original_width) // 2
                
                processed_data[start_y : start_y + original_height,
                               start_x : start_x + original_width] = original_data
            else:
                # Reescalar Lanczos para imágenes grandes o de tamaño similar
                zoom_factor_h = target_resolution / original_height
                zoom_factor_w = target_resolution / original_width

                # No necesitamos convertir a float32 aquí porque ya están normalizadas (float32)
                zoomed_data = zoom(original_data, (zoom_factor_h, zoom_factor_w), order=5) 
                
                # Aseguramos que la imagen reescalada tenga exactamente la resolución objetivo
                current_h, current_w = zoomed_data.shape
                
                if current_h != target_resolution or current_w != target_resolution:
                    final_zoomed_data = np.zeros((target_resolution, target_resolution), dtype=original_dtype)
                    
                    insert_h = min(target_resolution, current_h)
                    insert_w = min(target_resolution, current_w)

                    start_y_zoom = (target_resolution - insert_h) // 2
                    start_x_zoom = (target_resolution - insert_w) // 2

                    src_start_y = (current_h - insert_h) // 2
                    src_start_x = (current_w - insert_w) // 2

                    final_zoomed_data[start_y_zoom : start_y_zoom + insert_h,
                                      start_x_zoom : start_x_zoom + insert_w] = \
                                      zoomed_data[src_start_y : src_start_y + insert_h,
                                                  src_start_x : src_start_x + insert_w]
                    processed_data = final_zoomed_data.astype(original_dtype)
                else:
                    processed_data = zoomed_data.astype(original_dtype)
            
            # Guardar el archivo .npy procesado
            np.save(output_path, processed_data)

        except Exception as e:
            print(f"Error procesando {filename} (reescalado Lanczos): {e}")

# Directorios de salida para los datos normalizados (.npy)
normalized_base_dir = "../data/log/normalized/"
dud_normalized_dir = os.path.join(normalized_base_dir, "dud")
acs_normalized_dir = os.path.join(normalized_base_dir, "acs")

# Define los directorios de salida para las imágenes .npy procesadas
# (después de normalización y reescalado/padding)
processed_dud_npy_dir = os.path.join(normalized_base_dir,"pading608", "dud")
processed_acs_npy_dir = os.path.join(normalized_base_dir,"pading608", "acs")

# Definir resoluciones objetivo (las mismas que antes)
target_dud_resolution = 152
target_acs_resolution = 608

print("\n--- PASO 2: Aplicando Padding/Reescalado Lanczos a datos .npy normalizados ---")

# Procesar las imágenes DUD normalizadas
process_normalized_npy(dud_normalized_dir, processed_dud_npy_dir, target_dud_resolution)

# Procesar las imágenes ACS normalizadas
process_normalized_npy(acs_normalized_dir, processed_acs_npy_dir, target_acs_resolution)

print("\n🎉 ¡Procesamiento completo: normalización y reescalado Lanczos aplicados!")
print(f"Imágenes DUD procesadas guardadas en: {processed_dud_npy_dir}")
print(f"Imágenes ACS procesadas guardadas en: {processed_acs_npy_dir}")


--- PASO 2: Aplicando Padding/Reescalado Lanczos a datos .npy normalizados ---
Directorio de salida '../data/log/normalized/pading608\dud' creado.

Procesando imágenes .npy en '../data/log/normalized/dud' a 152x152 (padding/reescalado Lanczos)...


100%|███████████████████████████████████████████████████████████████████████████| 16114/16114 [00:17<00:00, 927.61it/s]


Directorio de salida '../data/log/normalized/pading608\acs' creado.

Procesando imágenes .npy en '../data/log/normalized/acs' a 608x608 (padding/reescalado Lanczos)...


100%|███████████████████████████████████████████████████████████████████████████| 16114/16114 [00:45<00:00, 351.68it/s]


🎉 ¡Procesamiento completo: normalización y reescalado Lanczos aplicados!
Imágenes DUD procesadas guardadas en: ../data/log/normalized/pading608\dud
Imágenes ACS procesadas guardadas en: ../data/log/normalized/pading608\acs


In [4]:
# Aumento de datos

In [5]:
## Rotación y flip:

In [3]:
import os
import numpy as np
from tqdm import tqdm

# Directorios
input_dud_dir = "../data/log/normalized/pading608/dud/"
input_acs_dir = "../data/log/normalized/pading608/acs/"

output_dud_dir = "../data/log/AugmentationLow608/dud/"
output_acs_dir = "../data/log/AugmentationLow608/acs/"

os.makedirs(output_dud_dir, exist_ok=True)
os.makedirs(output_acs_dir, exist_ok=True)

# Aumentos (función, sufijo)
augmentations = [
    (lambda x: np.rot90(x, 1), "rot90"),
    (lambda x: np.rot90(x, 2), "rot180"),
    (lambda x: np.rot90(x, 3), "rot270"),
    (lambda x: np.flipud(x), "flipud"),
    (lambda x: np.fliplr(x), "fliplr"),
    (lambda x: np.flipud(np.rot90(x, 1)), "flipud_rot90"),
    (lambda x: np.fliplr(np.rot90(x, 1)), "fliplr_rot90"),
]

original_count = 0
augmented_count = 0

print("🔄 Generando aumentos de datos y copiando originales...")

# Buscar pares válidos
dud_files = [f for f in os.listdir(input_dud_dir) if f.endswith(".npy")]

for dud_file in tqdm(dud_files):
    base_name = dud_file.replace(".npy", "")
    acs_file = base_name + ".npy"

    dud_path = os.path.join(input_dud_dir, dud_file)
    acs_path = os.path.join(input_acs_dir, acs_file)

    if not os.path.exists(acs_path):
        print(f"⚠️ Falta el archivo ACS para {dud_file}")
        continue

    # Cargar datos
    dud_data = np.load(dud_path)
    acs_data = np.load(acs_path)

    # Guardar copia original
    np.save(os.path.join(output_dud_dir, f"{base_name}.npy"), dud_data)
    np.save(os.path.join(output_acs_dir, f"{base_name}.npy"), acs_data)
    original_count += 1

    # Aumentos
    for i, (aug_fn, suffix) in enumerate(augmentations, start=1):
        aug_dud = aug_fn(dud_data)
        aug_acs = aug_fn(acs_data)

        aug_dud_path = os.path.join(output_dud_dir, f"{base_name}_aug{i}_{suffix}.npy")
        aug_acs_path = os.path.join(output_acs_dir, f"{base_name}_aug{i}_{suffix}.npy")

        np.save(aug_dud_path, aug_dud)
        np.save(aug_acs_path, aug_acs)
        augmented_count += 1

# Estadísticas finales
total = original_count + augmented_count
print("\n✅ Aumento de datos completado.")
print(f"🔹 Pares originales: {original_count}")
print(f"🔸 Datos aumentados: {augmented_count}")
print(f"📦 Total final de archivos: {total}")


🔄 Generando aumentos de datos y copiando originales...


100%|████████████████████████████████████████████████████████████████████████████| 16114/16114 [26:10<00:00, 10.26it/s]


✅ Aumento de datos completado.
🔹 Pares originales: 16114
🔸 Datos aumentados: 112798
📦 Total final de archivos: 128912


In [4]:
# Procesado para adaptar los datos a la SRGAN

import os
import numpy as np

# Directorios con los archivos .npy
output_dud_dir = "../data/log/AugmentationLow608/dud/"
output_acs_dir = "../data/log/AugmentationLow608/acs/"

# Función para cargar y mostrar resolución de un archivo .npy
def check_resolution(dir_path):
    # Lista de archivos .npy
    npy_files = [f for f in os.listdir(dir_path) if f.endswith(".npy")]
    if not npy_files:
        print(f"No hay archivos .npy en {dir_path}")
        return

    # Seleccionamos el primero
    npy_path = os.path.join(dir_path, npy_files[0])
    img = np.load(npy_path)

    print(f"Imagen cargada de {npy_path}")
    print(f"Resolución: {img.shape}")  # Por ejemplo: (152, 152, 3)

# Mostrar resolución de una imagen en cada carpeta
check_resolution(output_dud_dir)
check_resolution(output_acs_dir)


Imagen cargada de ../data/log/AugmentationLow608/dud/n10_0_87915.npy
Resolución: (152, 152)
Imagen cargada de ../data/log/AugmentationLow608/acs/n10_0_87915.npy
Resolución: (608, 608)


In [3]:
## Zoom y deslizamiento

In [ ]:
import os
import numpy as np
from tqdm import tqdm

# Directorios de entrada
input_dud_dir = "../data/log/Augmentation/dud/"
input_acs_dir = "../data/log/Augmentation/acs/"

# Directorios de salida
output_dud_dir = "../data/log/AugZum/dud/"
output_acs_dir = "../data/log/AugZum/acs/"
os.makedirs(output_dud_dir, exist_ok=True)
os.makedirs(output_acs_dir, exist_ok=True)

# Parámetros de zoom (2 píxeles de margen)
crop_margin = 2
original_count = 0
augmented_count = 0

print("🔍 Generando aumentos por zoom con deslizamiento...")

dud_files = [f for f in os.listdir(input_dud_dir) if f.endswith(".npy")]

for file in tqdm(dud_files):
    base_name = file.replace(".npy", "")
    dud_path = os.path.join(input_dud_dir, file)
    acs_path = os.path.join(input_acs_dir, file)

    if not os.path.exists(acs_path):
        print(f"⚠️ Falta el archivo ACS para {file}")
        continue

    # Cargar imágenes
    dud = np.load(dud_path)
    acs = np.load(acs_path)

    h_dud, w_dud = dud.shape
    h_acs, w_acs = acs.shape

    # Guardar copia original
    np.save(os.path.join(output_dud_dir, file), dud)
    np.save(os.path.join(output_acs_dir, file), acs)
    original_count += 1

    # 4 crops con desplazamiento de 2 píxeles
    for idx, (dy, dx) in enumerate([(0, 0), (0, crop_margin), (crop_margin, 0), (crop_margin, crop_margin)], start=1):
        dud_crop = dud[dy:h_dud - crop_margin + dy, dx:w_dud - crop_margin + dx]
        acs_crop = acs[dy:h_acs - crop_margin + dy, dx:w_acs - crop_margin + dx]

        dud_out = os.path.join(output_dud_dir, f"{base_name}_zoom{idx}.npy")
        acs_out = os.path.join(output_acs_dir, f"{base_name}_zoom{idx}.npy")

        np.save(dud_out, dud_crop)
        np.save(acs_out, acs_crop)
        augmented_count += 1

# Resumen
total = original_count + augmented_count
print("\n✅ Zoom con deslizamiento completado.")
print(f"🔹 Originales procesados: {original_count}")
print(f"🔸 Zooms generados: {augmented_count}")
print(f"📦 Total final: {total}")

🔍 Generando aumentos por zoom con deslizamiento...


  1%|▍                                                                          | 825/128912 [01:33<5:27:12,  6.52it/s]

In [9]:
import os
import numpy as np
from collections import Counter
 
folder_path = "../data/AugZum/dud"  
 
shapes = []
 
for file in os.listdir(folder_path):
    if file.endswith(".npy"):
        path = os.path.join(folder_path, file)
        try:
            arr = np.load(path)
            shapes.append(arr.shape)
        except Exception as e:
            print(f"Error leyendo {file}: {e}")
 
# Mostrar cuántas imágenes hay de cada tamaño
conteo = Counter(shapes)
for shape, count in conteo.items():
    print(f"{shape}: {count} imágenes")

(16, 16): 22784 imágenes
(13, 13): 29624 imágenes
(26, 26): 6344 imágenes
(10, 11): 7076 imágenes
(14, 14): 27472 imágenes
(17, 16): 5376 imágenes
(31, 31): 3768 imágenes
(10, 10): 25392 imágenes
(22, 21): 3544 imágenes
(71, 72): 80 imágenes
(7, 8): 3216 imágenes
(15, 15): 22736 imágenes
(11, 11): 25160 imágenes
(8, 8): 13896 imágenes
(48, 48): 792 imágenes
(14, 13): 7008 imágenes
(30, 30): 4032 imágenes
(9, 9): 23664 imágenes
(11, 12): 6520 imágenes
(6, 7): 1264 imágenes
(18, 18): 14280 imágenes
(12, 13): 5560 imágenes
(31, 32): 820 imágenes
(22, 22): 14496 imágenes
(9, 8): 4672 imágenes
(19, 20): 3828 imágenes
(17, 17): 21712 imágenes
(18, 17): 4120 imágenes
(13, 14): 7008 imágenes
(14, 15): 6328 imágenes
(19, 18): 4580 imágenes
(12, 12): 31968 imágenes
(20, 20): 12968 imágenes
(20, 19): 3828 imágenes
(16, 17): 5376 imágenes
(8, 9): 4672 imágenes
(7, 7): 11304 imágenes
(16, 15): 5292 imágenes
(28, 28): 5168 imágenes
(25, 25): 7336 imágenes
(35, 35): 1968 imágenes
(41, 41): 1728 imáge

# Split en train, val y test

In [7]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
import shutil
from tqdm import tqdm

def split_data_into_sets(input_dirs, output_base_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, random_state=42):
    """
    Divide los datos en conjuntos de entrenamiento, validación y prueba, manteniendo la estructura de directorios.
    
    Args:
        input_dirs (dict): Diccionario con las rutas de entrada para cada clase (ej. {'dud': path, 'acs': path})
        output_base_dir (str): Directorio base donde se crearán los subdirectorios train/val/test
        train_ratio (float): Proporción para el conjunto de entrenamiento (por defecto 0.7)
        val_ratio (float): Proporción para el conjunto de validación (por defecto 0.15)
        test_ratio (float): Proporción para el conjunto de prueba (por defecto 0.15)
        random_state (int): Semilla para reproducibilidad (por defecto 42)
    """
    # Verificar que los ratios sumen 1
    assert abs((train_ratio + val_ratio + test_ratio) - 1.0) < 1e-6, "Los ratios deben sumar 1"
    
    # Crear directorios de salida
    train_dir = os.path.join(output_base_dir, 'train')
    val_dir = os.path.join(output_base_dir, 'val')
    test_dir = os.path.join(output_base_dir, 'test')
    
    for dir_path in [train_dir, val_dir, test_dir]:
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
            for class_name in input_dirs.keys():
                os.makedirs(os.path.join(dir_path, class_name))
    
    print("\nDividiendo datos en conjuntos train/val/test...")
    
    for class_name, input_dir in input_dirs.items():
        # Obtener lista de archivos
        files = [f for f in os.listdir(input_dir) if f.endswith('.npy')]
        files.sort()  # Para reproducibilidad
        
        if not files:
            print(f"Advertencia: No se encontraron archivos .npy en {input_dir}")
            continue
        
        # Dividir en train y temp (val + test)
        train_files, temp_files = train_test_split(
            files, 
            test_size=(1 - train_ratio), 
            random_state=random_state
        )
        
        # Dividir temp en val y test
        val_test_ratio = val_ratio / (val_ratio + test_ratio)
        val_files, test_files = train_test_split(
            temp_files, 
            test_size=(1 - val_test_ratio), 
            random_state=random_state
        )
        
        print(f"\nClase {class_name}:")
        print(f"  Total: {len(files)} imágenes")
        print(f"  Train: {len(train_files)} imágenes ({len(train_files)/len(files):.1%})")
        print(f"  Val:   {len(val_files)} imágenes ({len(val_files)/len(files):.1%})")
        print(f"  Test:  {len(test_files)} imágenes ({len(test_files)/len(files):.1%})")
        
        # Función para copiar archivos
        def copy_files(files, dest_dir):
            for file in tqdm(files, desc=f"Copiando {class_name} a {os.path.basename(dest_dir)}"):
                src_path = os.path.join(input_dir, file)
                dest_path = os.path.join(dest_dir, class_name, file)
                shutil.copy2(src_path, dest_path)
        
        # Copiar archivos a sus directorios correspondientes
        copy_files(train_files, train_dir)
        copy_files(val_files, val_dir)
        copy_files(test_files, test_dir)

# Directorios de entrada (los mismos que usaste en tu código anterior)
processed_dud_npy_dir = "../data/log/AugmentationLow608/dud/"
processed_acs_npy_dir = "../data/log/AugmentationLow608/acs/"

#processed_dud_npy_dir = os.path.join(normalized_base_dir, "pading", "dud")
#processed_acs_npy_dir = os.path.join(normalized_base_dir, "pading", "acs")

# Directorio de salida para los conjuntos divididos
split_data_dir = os.path.join("../data/log/AugmentationLow608/", "split_data")
#split_data_dir = os.path.join(normalized_base_dir, "split_data")

# Definir los directorios de entrada para cada clase
input_dirs = {
    'dud': processed_dud_npy_dir,
    'acs': processed_acs_npy_dir
}

# Dividir los datos (70% train, 15% val, 15% test)
split_data_into_sets(
    input_dirs=input_dirs,
    output_base_dir=split_data_dir,
    train_ratio=0.8,
    val_ratio=0.10,
    test_ratio=0.10,
    random_state=42
)

print("\n🎉 ¡División de datos completada!")
print(f"Conjunto train: {os.path.join(split_data_dir, 'train')}")
print(f"Conjunto val:   {os.path.join(split_data_dir, 'val')}")
print(f"Conjunto test:  {os.path.join(split_data_dir, 'test')}")


Dividiendo datos en conjuntos train/val/test...

Clase dud:
  Total: 128912 imágenes
  Train: 103129 imágenes (80.0%)
  Val:   12891 imágenes (10.0%)
  Test:  12892 imágenes (10.0%)


Copiando dud a test: 100%|██████████████████████████████████████████████████████| 12892/12892 [00:19<00:00, 665.64it/s]



Clase acs:
  Total: 128912 imágenes
  Train: 103129 imágenes (80.0%)
  Val:   12891 imágenes (10.0%)
  Test:  12892 imágenes (10.0%)


Copiando acs a test: 100%|██████████████████████████████████████████████████████| 12892/12892 [01:09<00:00, 185.82it/s]


🎉 ¡División de datos completada!
Conjunto train: ../data/log/AugmentationLow608/split_data\train
Conjunto val:   ../data/log/AugmentationLow608/split_data\val
Conjunto test:  ../data/log/AugmentationLow608/split_data\test


In [8]:
import os

def check_matching_pairs(base_dir):
    """
    Verifica que todos los archivos dud tengan su correspondiente archivo acs en el mismo conjunto
    y viceversa.
    
    Args:
        base_dir (str): Directorio base que contiene las subcarpetas train/val/test
    """
    sets = ['train', 'val', 'test']
    problems_found = False
    
    for set_name in sets:
        set_path = os.path.join(base_dir, set_name)
        dud_dir = os.path.join(set_path, 'dud')
        acs_dir = os.path.join(set_path, 'acs')
        
        # Obtener listas de archivos (sin la extensión .npy)
        dud_files = set([f.replace('.npy', '') for f in os.listdir(dud_dir) if f.endswith('.npy')])
        acs_files = set([f.replace('.npy', '') for f in os.listdir(acs_dir) if f.endswith('.npy')])
        
        # Encontrar diferencias
        dud_without_acs = dud_files - acs_files
        acs_without_dud = acs_files - dud_files
        
        if dud_without_acs:
            print(f"⚠️ En {set_name}: {len(dud_without_acs)} archivos dud sin pareja acs")
            print(f"   Ejemplos: {list(dud_without_acs)[:5]}")
            problems_found = True
            
        if acs_without_dud:
            print(f"⚠️ En {set_name}: {len(acs_without_dud)} archivos acs sin pareja dud")
            print(f"   Ejemplos: {list(acs_without_dud)[:5]}")
            problems_found = True
    
    if not problems_found:
        print("✅ Todas las parejas dud-acs están correctamente emparejadas en todos los conjuntos")

# Directorio base donde están train/val/test
normalized_base_dir = "../data/log/AugmentationLow608/"
split_data_dir = os.path.join(normalized_base_dir, "split_data")
check_matching_pairs(split_data_dir)

✅ Todas las parejas dud-acs están correctamente emparejadas en todos los conjuntos


In [9]:
# convierte los archivos .npy a imágenes .png
import os
import numpy as np
import matplotlib.pyplot as plt

def convert_npy_to_png_and_cleanup(base_dir):
    """
    Convierte archivos .npy a .png en cada carpeta (dud/acs) dentro de train/val/test
    y elimina los .npy después de convertirlos.
    
    Args:
        base_dir (str): Directorio base que contiene las subcarpetas train/val/test
    """
    sets = ['train', 'val', 'test']
    subfolders = ['dud', 'acs']
    
    for set_name in sets:
        for subfolder in subfolders:
            folder_path = os.path.join(base_dir, set_name, subfolder)
            if not os.path.exists(folder_path):
                print(f"❌ Carpeta no encontrada: {folder_path}")
                continue

            for file in os.listdir(folder_path):
                if file.endswith('.npy'):
                    npy_path = os.path.join(folder_path, file)
                    png_filename = file.replace('.npy', '.png')
                    png_path = os.path.join(folder_path, png_filename)
                    
                    try:
                        data = np.load(npy_path)
                        
                        # Normalizar si está en [0,1]
                        if data.dtype != np.uint8:
                            data = (data * 255).clip(0, 255).astype(np.uint8)

                        plt.imsave(png_path, data, cmap='gray' if data.ndim == 2 else None)
                        os.remove(npy_path)
                        #print(f"✅ Convertido y eliminado: {npy_path} -> {png_path}")
                    
                    except Exception as e:
                        print(f"❌ Error procesando {npy_path}: {e}")

# Uso
normalized_base_dir = "../data/log/AugmentationLow608/"
split_data_dir = os.path.join(normalized_base_dir, "split_data")
convert_npy_to_png_and_cleanup(split_data_dir)
print("✅ Proceso TERMINADO")

C:\Users\Miguel\AppData\Local\Temp\ipykernel_20836\2766978106.py:35: RuntimeWarning: invalid value encountered in cast
  data = (data * 255).clip(0, 255).astype(np.uint8)


❌ Error procesando ../data/log/AugmentationLow608/split_data\train\acs\n25_250_62252_aug4_flipud.npy: [WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: '../data/log/AugmentationLow608/split_data\\train\\acs\\n25_250_62252_aug4_flipud.npy'
✅ Proceso TERMINADO
